# Add Flexibility to Calculation of Allocations

Thu 13 Sep, 2018

In [1]:
import numpy as np
import pandas as pd

## Create DataFrames with test data

In [20]:
prev_equity_df = pd.DataFrame([
    [1, 'Alice', 0, 0, 0],
    [2, 'Bob', 125, 0, 0],
    [3, 'Candice', 125, 3000, 200],
    [4, 'Darwin', 0, 0, 0],
    [5, 'Eve', 125, 1000, 100],
    [6, 'Federico', 125, 0, 700],
    [7, 'Gabi', 125, 0, 0],
    [8, 'HAL', 125, 0, 0],
    [9, 'Irene', 125, 10000, 0]
], columns = ['member_id', 'name', 'membership', 'preferred', 'other'])
prev_equity_df['equity'] = prev_equity_df['membership'] + prev_equity_df['preferred'] + prev_equity_df['other']
prev_equity_df

,member_id,name,membership,preferred,other,equity
0,1,Alice,0,0,0,0
1,2,Bob,125,0,0,125
2,3,Candice,125,3000,200,3325
3,4,Darwin,0,0,0,0
4,5,Eve,125,1000,100,1225
5,6,Federico,125,0,700,825
6,7,Gabi,125,0,0,125
7,8,HAL,125,0,0,125
8,9,Irene,125,10000,0,10125


In [21]:
contributions_df = pd.DataFrame([
    [1, "1/3/2017", 125, 'membership'],
    [2, "1/31/2017", 400, 'other'],
    [1, "2/20/2017", 5000, 'preferred'],
    [3, "3/15/2017", -2000, 'preferred'],
    [1, "4/13/2017", 500, 'other'],
    [2, "4/13/2017", 7500, 'preferred'],
    [4, "6/24/2017", 125, 'mebership'],
    [5, "7/17/2017", 3000, "preferred"],
    [6, "8/22/2017", -700, 'other'],
    [4, "10/18/2017", 20000, 'preferred'],
    [6, "11/11/2017", 1500, 'preferred']
], columns = ['member_id','date','amount','type'])
contributions_df

,member_id,date,amount,type
0,1,1/3/2017,125,membership
1,2,1/31/2017,400,other
2,1,2/20/2017,5000,preferred
3,3,3/15/2017,-2000,preferred
4,1,4/13/2017,500,other
5,2,4/13/2017,7500,preferred
6,4,6/24/2017,125,mebership
7,5,7/17/2017,3000,preferred
8,6,8/22/2017,-700,other
9,4,10/18/2017,20000,preferred


## Collect functions from dividends notebook

Change percents to proportions.

In [13]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    contributions_df = contributions_df.copy()
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    contributions_df['patronage'] = patronage
    return contributions_df

def compute_new_patronage(contributions_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    contributions_df = compute_transaction_patronage(contributions_df)
    return contributions_df[['member_id','patronage']].groupby(by='member_id').sum()

def compute_patronage(prev_equity_df, contributions_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = prev_equity_df.set_index('member_id')[['name', 'equity']]
    patronage_df.rename(columns={'equity': 'old_patronage'}, inplace=True)
    
    patronage_df['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
    # If there were members with no contributions this year, set their new patronage to 0 (would be NaN).
    patronage_df.fillna(0, inplace=True)
    
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    
    return patronage_df

def compute_dividends(patronage_df, profit, proportion_individual=0.5):
    """Compute each member's dividend based on patronage for the year.
    """ 
    dividend_df = patronage_df[['name', 'proportionate_patronage']].copy()
    
    #Compute individual patronage allocations
    dividend_df['dividend'] = np.round(
        dividend_df['proportionate_patronage'] * profit * proportion_individual, 2)
    
    # To account for rounding amounts to the nearest cent, we add up the individual dividends
    # to get the actual amount allocated to individual net income. Then we subtract this amount
    # from the total profit to get the collective net income.
    indiv_profit = dividend_df['dividend'].sum()
    collective_profit = profit - indiv_profit
    
    # We reserve member_id=0 for the collective account (or we could simply use names as keys)
    dividend_df.loc[0] = pd.Series({
        'name': 'CollectiveAcct',
        'proportionate_patronage': collective_profit / indiv_profit,
        'dividend': collective_profit
    })
    return dividend_df

## Auxiliary functions

In [18]:
def compute_total_patronage(patronage_df):
    """Computes the total patronage for the year, i.e. the time-averaged amount of equity.
    """
    return patronage_df['patronage'].sum()

def estimate_indiv_collective_profit(profit, proportion_individual):
    """Estimate the individual and collective net income from the total net income.
    This may be different from the actual individual and collective amounts because
    of rounding individual dividends to the nearest cent.
    """
    indiv_profit = np.round(profit * proportion_individual, 2)
    collective_profit = profit - indiv_profit
    return indiv_profit, collective_profit

In [22]:
patronage_df = compute_patronage(prev_equity_df, contributions_df)
patronage_df

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,4799.657534,4799.657534,0.158101
2,Bob,125,5771.232877,5896.232877,0.194222
3,Candice,3325,-1600.000000,1725.000000,0.056822
4,Darwin,0,4175.000000,4175.000000,0.137525
5,Eve,1225,1380.821918,2605.821918,0.085836
6,Federico,825,-43.561644,781.438356,0.025741
7,Gabi,125,0.000000,125.000000,0.004118
8,HAL,125,0.000000,125.000000,0.004118
9,Irene,10125,0.000000,10125.000000,0.333518


In [23]:
patronage_df.sum()

name                       AliceBobCandiceDarwinEveFedericoGabiHALIrene
old_patronage                                                     15875
new_patronage                                                   14483.2
patronage                                                       30358.2
proportionate_patronage                                               1
dtype: object

In [24]:
profit = 1729.45
dividend_df = compute_dividends(patronage_df, profit)
dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.158101,136.71
2,Bob,0.194222,167.95
3,Candice,0.056822,49.14
4,Darwin,0.137525,118.92
5,Eve,0.085836,74.22
6,Federico,0.025741,22.26
7,Gabi,0.004118,3.56
8,HAL,0.004118,3.56
9,Irene,0.333518,288.40


In [25]:
estimate_indiv_collective_profit(profit, 0.5)

(864.72000000000003, 864.73000000000002)